In [52]:
import numpy as np
import pandas as pd

In [53]:
def CW_PF(Re, D, ks):

    # Parámetros de la solución
    err = 1e10
    tol = 1e-12
    f0 = 0.02
    count = 1

    # Iterar
    while err > tol:
    
        # Calculando f1
        f1 = (2 * np.log10(ks / D / 3.7 + 2.51 / Re / np.sqrt(f0))) ** -2

        # Calcular el error
        err = np.abs((f1 - f0) / f0)

        print(f'Iteración: {count}\tf={f1:.7f}\tErr={err:.7f}')

        # Actualizo valores
        f0 = f1
        count += 1

    return f1

def CW_NR(Re, D, ks):

    # Parámetros de solver
    err = 1e10
    tol = 1e-12
    x0 = 0.02 ** -2
    # count = 1
    K = 2 * 2.51 / np.log(10)

    # Definir funciones rápidas
    gx = lambda x, D, ks, Re: x + 2 * np.log10(ks / D / 3.7 + 2.51 * x / Re)
    gpx = lambda x, D, ks, Re: 1 + K / (ks / D / 3.7 + 2.51 * x /Re) / Re

    # Iterando hasta llegar a la solución
    while err > tol:

        # Aplicando NR
        x1 = x0 - gx(x0, D, ks, Re) / gpx(x0, D, ks, Re)

        # Calculo error
        err = np.abs((x1 - x0) / x0)

        # print(f'Iteración: {count}\tf={x1 ** -2:.7f}\tErr={err:.7f}')

        # Actualizo valores
        x0 = x1
        # count += 1

    return x1 ** -2


In [54]:
print('Punto fijo:')
CW_PF(1350000, 300, 0.0015)
print('Newton-Raphson:')
CW_NR(1350000, 300, 0.0015)

Punto fijo:
Iteración: 1	f=0.0106779	Err=0.4661052
Iteración: 2	f=0.0112528	Err=0.0538436
Iteración: 3	f=0.0112025	Err=0.0044754
Iteración: 4	f=0.0112068	Err=0.0003830
Iteración: 5	f=0.0112064	Err=0.0000327
Iteración: 6	f=0.0112064	Err=0.0000028
Iteración: 7	f=0.0112064	Err=0.0000002
Iteración: 8	f=0.0112064	Err=0.0000000
Iteración: 9	f=0.0112064	Err=0.0000000
Iteración: 10	f=0.0112064	Err=0.0000000
Iteración: 11	f=0.0112064	Err=0.0000000
Iteración: 12	f=0.0112064	Err=0.0000000
Iteración: 13	f=0.0112064	Err=0.0000000
Newton-Raphson:


0.011206424816678637

In [55]:
# Resolver un problema de tuberías simples
Dn = 0.3                                  # Diam nominal (m)
Di = 0.293                                # Diam int (m)
ks = 1.5e-6                               # Rugos (m)
L = 730                                   # Long (m)
Dz = 43.5                                 # Dif alturas (m)
SK = 11.8                                 # K de accesorios
nu = 1.007e-6                             # Visocsidad cinem agua (m2/s)

# Parámetros numéricos
err = 1e10
tol = 1e-10
count = 1
f0 = 0.05

# Para guardar info
Vf = []
Vv = []
VRe = []
Verr = []

while err > tol:

    # Calcular velocidad
    u = np.sqrt(Dz * 19.62 / (f0 * L / Di + SK))

    # Calculo Re
    Re = u * Di / nu

    # Calculo f1
    f1 = CW_NR(Re, Di, ks)

    # print(f'Iter: {count}\tu(m/s): {u:.3f}\tf: {f1:.5f}')

    err = np.abs((f1 - f0) / f0)

    # Llenar vectores
    Vf.append(f0)
    Vv.append(u)
    VRe.append(Re)
    Verr.append(err)

    # Actualizo valores
    f0 = f1
    count += 1

In [56]:
Datos = pd.DataFrame({
    'f': np.round(Vf, 8),
    'v (m/s)': np.round(Vv, 4),
    'Re': np.round(VRe, 2),
    'Err_rel': np.round(Verr, 6)
})

Datos['Q (m3/s)'] = np.round(Datos['v (m/s)'] * np.pi * Di ** 2 / 4, 4)
Datos['hf (m)'] = (Datos['f'] * L / Di) * Datos['v (m/s)'] ** 2 / 19.62
Datos['hf (m)'] = np.round(Datos['hf (m)'], 4)
Datos['hk (m)'] = SK * Datos['v (m/s)'] ** 2 / 19.62
Datos['hk (m)'] = np.round(Datos['hk (m)'], 4)
Datos['Ht (m)'] = Datos['hk (m)'] + Datos['hf (m)']

Datos

,f,v (m/s),Re,Err_rel,Q (m3/s),hf (m),hk (m),Ht (m)
0,0.050000,2.5017,727892.89,0.751999,0.1687,39.7371,3.7640,43.5011
1,0.012400,4.4710,1300909.53,0.090710,0.3015,31.4767,12.0224,43.4991
2,0.011275,4.6254,1345828.89,0.005333,0.3119,30.6324,12.8671,43.4995
3,0.011215,4.6341,1348363.18,0.000296,0.3125,30.5838,12.9156,43.4994
4,0.011212,4.6346,1348503.32,0.000016,0.3125,30.5813,12.9184,43.4997
5,0.011212,4.6346,1348511.06,0.000001,0.3125,30.5808,12.9184,43.4992
6,0.011212,4.6346,1348511.48,0.000000,0.3125,30.5808,12.9184,43.4992
7,0.011212,4.6346,1348511.51,0.000000,0.3125,30.5808,12.9184,43.4992
8,0.011212,4.6346,1348511.51,0.000000,0.3125,30.5808,12.9184,43.4992
9,0.011212,4.6346,1348511.51,0.000000,0.3125,30.5808,12.9184,43.4992


In [57]:
def TS_CW(Di, ks, Dz, L, SK, nu):

    # Parámetros numéricos
    err = 1e10
    tol = 1e-10
    f0 = 0.05

    # Iterar para hallar la velocidad y el f del problema
    while err > tol:

        # Calcular velocidad
        u = np.sqrt(Dz * 19.62 / (f0 * L / Di + SK))

        # Calculo Re
        Re = u * Di / nu

        # Calculo f1
        f1 = CW_NR(Re, Di, ks)

        # print(f'Iter: {count}\tu(m/s): {u:.3f}\tf: {f1:.5f}')

        err = np.abs((f1 - f0) / f0)

        # Actualizo valores
        f0 = f1

    return u, f1, Re

In [58]:
# Datos (pasando a m de una vez)
Dn = [75, 100, 150, 200, 250, 300]
Di = np.array([80.42, 103.42, 152.22, 198.48, 247.09, 293.07]) / 1000
ks_h = 0.15e-3
ks_pvc = 0.0015e-3

Q = 0.06
L = 890
Dz = 15.2
SK = 11.9
nu = 1.14e-6

# Arreglos para guardar info
vP = []
qP = []
fP = []
ReP = []

for diam in Di:

    u, f1, Re = TS_CW(diam, ks_pvc, Dz, L, SK, nu)

    vP.append(u)
    qP.append(u * np.pi * diam ** 2 / 4)
    fP.append(f1)
    ReP.append(Re)

Datos_P = pd.DataFrame({
    'Dn (mm)': Dn,
    'Di (mm)': Di * 1000, 
    'Q (m3/s)': np.round(qP, 5)

})

Datos_P['Q (L/s)'] = np.round(Datos_P['Q (m3/s)'] * 1000, 4)

cumple = ['Cumple' if qin >= Q * 1000 else 'No cumple'\
          for qin in Datos_P['Q (L/s)']]

Datos_P['Cumple'] = cumple

Datos_P

,Dn (mm),Di (mm),Q (m3/s),Q (L/s),Cumple
0,75,80.42,0.00591,5.91,No cumple
1,100,103.42,0.01144,11.44,No cumple
2,150,152.22,0.03120,31.20,No cumple
3,200,198.48,0.06152,61.52,Cumple
4,250,247.09,0.10700,107.00,Cumple
5,300,293.07,0.16384,163.84,Cumple


In [59]:
# Datos (pasando a m de una vez)
Dn = [75, 100, 150, 200, 250, 300]
Di = np.array([80.42, 103.42, 152.22, 198.48, 247.09, 293.07]) / 1000
ks_h = 0.15e-3
ks_pvc = 0.0015e-3

Q = 0.06
L = 890
Dz = 15.2
SK = 11.9
nu = 1.14e-6

# Arreglos para guardar info
vP = []
qP = []
fP = []
ReP = []

for diam in Di:

    u, f1, Re = TS_CW(diam, ks_h, Dz, L, SK, nu)

    vP.append(u)
    qP.append(u * np.pi * diam ** 2 / 4)
    fP.append(f1)
    ReP.append(Re)

Datos_P = pd.DataFrame({
    'Dn (mm)': Dn,
    'Di (mm)': Di * 1000, 
    'Q (m3/s)': np.round(qP, 5)

})

Datos_P['Q (L/s)'] = np.round(Datos_P['Q (m3/s)'] * 1000, 4)

cumple = ['Cumple' if qin >= Q * 1000 else 'No cumple'\
          for qin in Datos_P['Q (L/s)']]

Datos_P['Cumple'] = cumple

Datos_P

,Dn (mm),Di (mm),Q (m3/s),Q (L/s),Cumple
0,75,80.42,0.00513,5.13,No cumple
1,100,103.42,0.00992,9.92,No cumple
2,150,152.22,0.02709,27.09,No cumple
3,200,198.48,0.05358,53.58,No cumple
4,250,247.09,0.09358,93.58,Cumple
5,300,293.07,0.14391,143.91,Cumple
